In [ ]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import run_full_analysis 

### Define task function

In [ ]:
# ISO3 = "FRA"
# ANALYSIS = "main"

# N_CHAINS = 32
# N_OPTI_SEARCHES = 8
# OPTI_BUDGET = 7000

# METROPOLIS_TUNE = 2000
# METROPOLIS_DRAWS = 10000
# METROPOLIS_METHOD = "DEMetropolis"

# FULL_RUNS_SAMPLES = 1000
# BURN_IN = 5000

In [ ]:
ISO3 = "FRA"
ANALYSIS = "main"

N_CHAINS = 32
N_OPTI_SEARCHES = 8
OPTI_BUDGET = 700

METROPOLIS_TUNE = 200
METROPOLIS_DRAWS = 1000
METROPOLIS_METHOD = "DEMetropolis"

FULL_RUNS_SAMPLES = 100
BURN_IN = 500

### Check that configuration is relevant

In [ ]:
assert N_CHAINS % N_OPTI_SEARCHES == 0
assert (METROPOLIS_DRAWS - BURN_IN) * N_CHAINS >= FULL_RUNS_SAMPLES
assert METROPOLIS_METHOD in ("DEMetropolis", "DEMetropolisZ")

In [ ]:
def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        analysis=analysis, 
        opti_params={'n_searches': N_OPTI_SEARCHES, 'num_workers': 8, 'parallel_opti_jobs': 4, 'warmup_iterations': 0, 'search_iterations': OPTI_BUDGET, 'init_method': 'LHS'},
        mcmc_params={'draws': METROPOLIS_DRAWS, 'tune': METROPOLIS_TUNE, 'cores': N_CHAINS, 'chains': N_CHAINS, 'method': METROPOLIS_METHOD},
        full_run_params={'samples': FULL_RUNS_SAMPLES, 'burn_in': BURN_IN},
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [ ]:
mspec = springboard.EC2MachineSpec(N_CHAINS, 4, "compute")
task_kwargs = {
    "iso3": ISO3,
    "analysis": ANALYSIS
}
tspec = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [ ]:
analysis_title = "test_parallel"
config_str = f"_{ANALYSIS}_LHS{N_OPTI_SEARCHES}_opt{OPTI_BUDGET}_mc{METROPOLIS_TUNE}n{METROPOLIS_DRAWS}_DE"
if METROPOLIS_METHOD == "DEMetropolisZ":
    config_str += "Z"

run_path = springboard.launch.get_autumn_project_run_path("school_project", ISO3, analysis_title + config_str)
run_path

In [ ]:
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path)

In [ ]:
from time import sleep

update_freq = 30
status = runner.s3.get_status()
print(status)

while status in ['INIT', 'LAUNCHING']:
    sleep(update_freq)
    status = runner.s3.get_status()
print(status)

while status in ['RUNNING']:
    sleep(update_freq)
    status = runner.s3.get_status()
print(status)

In [ ]:
from autumn.core.runs import ManagedRun

In [ ]:
# run_path = 'projects/school_project/FRA/2023-06-30T1440-main_LHS8_opti10k_mcmc10k_DE_19p'

In [ ]:
mr = ManagedRun(run_path)

In [ ]:
for f in mr.remote.list_contents():
    mr.remote.download(f)